In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import os
import sys
import cv2
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision import models
import json

path="/content/drive/My Drive/IFT6759/Ebird"
os.chdir(path)
os.listdir(path)
pthfile = r'checkpoint_498_240.pth'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
class GradCAM(object):
    def __init__(self, net, layer_name):
        self.net = net
        self.layer_name = layer_name
        self.feature = None
        self.gradient = None
        self.net.eval()
        self.handlers = []
        self._register_hook()

    def _get_features_hook(self, module, input, output):
        self.feature = output
        print("feature shape:{}".format(output.size()))

    def _get_grads_hook(self, module, input_grad, output_grad):
        """
        :param input_grad: tuple, input_grad[0]: None
                                   input_grad[1]: weight
                                   input_grad[2]: bias
        :param output_grad:tuple,length = 1
        :return:
        """
        self.gradient = output_grad[0]

    def _register_hook(self):
        for (name, module) in self.net.named_modules():
            if name == self.layer_name:
                self.handlers.append(module.register_forward_hook(self._get_features_hook))
                self.handlers.append(module.register_backward_hook(self._get_grads_hook))

    def remove_handlers(self):
        for handle in self.handlers:
            handle.remove()

    def __call__(self, inputs, index):
        """
        :param inputs: [1,3,H,W]
        :param index: class id
        :return:
        """
        self.net.zero_grad()
        output = self.net(inputs)  # [1,num_classes]
        if index is None:
            index = np.argmax(output.cpu().data.numpy())
        target = output[0][index]
        target.backward()

        gradient = self.gradient[0].cpu().data.numpy()  # [C,H,W]
        weight = np.mean(gradient, axis=(1, 2))  # [C]

        feature = self.feature[0].cpu().data.numpy()  # [C,H,W]

        cam = feature * weight[:, np.newaxis, np.newaxis]  # [C,H,W]
        cam = np.sum(cam, axis=0)  # [H,W]
        cam = np.maximum(cam, 0)  # ReLU

        # normalize
        cam -= np.min(cam)
        cam /= np.max(cam)
        # resize to 256*256
        cam = cv2.resize(cam, (256, 256))
        return cam

In [ ]:
import torch
from torch import nn
import numpy as np


class GuidedBackPropagation(object):

    def __init__(self, net):
        self.net = net
        for (name, module) in self.net.named_modules():
            if isinstance(module, nn.ReLU):
                module.register_backward_hook(self.backward_hook)
        self.net.eval()

    @classmethod
    def backward_hook(cls, module, grad_in, grad_out):
        """
        :param module:
        :param grad_in: tuple, length = 1
        :param grad_out: tuple, length = 1
        :return: tuple(new_grad_in,)
        """
        return torch.clamp(grad_in[0], min=0.0),

    def __call__(self, inputs, index=None):
        """
        :param inputs: [1,3,H,W]
        :param index: class_id
        :return:
        """
        self.net.zero_grad()
        output = self.net(inputs)  # [1,num_classes]
        if index is None:
            index = np.argmax(output.cpu().data.numpy())
        target = output[0][index]

        target.backward()

        return inputs.grad[0]  # [3,H,W]

In [ ]:
import argparse
import os
import re

import cv2
import numpy as np
import torch
from skimage import io
from torch import nn
from torchvision import models


import matplotlib.pyplot as plt
from scipy import misc
import scipy

def get_net(net_name, weight_path=None):
    """
    Get the model by network name
    :param net_name: network name
    :param weight_path: The path of trained weights
    :return:
    """
    Ebird_net = torch.load(pthfile,map_location=torch.device('cpu'))
    layers = list(Ebird_net["model_state_dict"].keys())
    for i in layers: 
      Ebird_net["model_state_dict"][i.replace('backbone.', '')] =  Ebird_net["model_state_dict"].pop(i)

    model = models.resnet50(pretrained=False)
    conv1 = torch.nn.Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model.conv1 = conv1
    fc_features = model.fc.in_features 
    model.fc = torch.nn.Linear(2048, 2048)
    model.classification_head = torch.nn.Sequential(
        torch.nn.Linear(2128, 2128),
        torch.nn.ReLU(),
        torch.nn.Linear(2128, 2128),
        torch.nn.ReLU(),
        torch.nn.Linear(2128, 2128))
    model.load_state_dict(Ebird_net['model_state_dict'], strict=False)
    return model


def get_last_conv_name(net): 
    """
    Gets the name of the last convolutional layer of the network
    :param net:
    :return:
    """
    layer_name = None
    for name, m in net.named_modules():
        if isinstance(m, nn.Conv2d):
            layer_name = name
    print(layer_name)
    return layer_name


def prepare_input(image): 
    image = image.copy().astype('float32')

    # normalize
    means = np.array([887.4186,  929.5382,  688.8167, 2818.0081])
    stds = np.array([873.2404,  734.9333,  734.1111, 1351.0328])
    image -= means
    image /= stds

    image = np.ascontiguousarray(np.transpose(image, (2, 0, 1)))  # channel first
    image = image[np.newaxis, ...]  # adding batch dimension

    return torch.tensor(image, requires_grad=True)


def gen_cam(image, mask): 
    """
    Generated CAM figure
    :param image: [H,W,C], original image
    :param mask: [H,W],range 0~1
    :return: tuple(cam,heatmap)
    """
    # mask to heatmap
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    #heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    heatmap = np.float32(heatmap) / 255
    heatmap = heatmap[..., ::-1]  # gbr to rgb

    # Merge heatMap to original image
    nir_heatmap = image[:,:,2] / np.max(image[:,:,2])
    heatmap_nir = np.insert(heatmap, 0, values=nir_heatmap, axis=2)
    cam = cv2.addWeighted(heatmap * 255, 0.8, image[:,:,:3], 0.05, 0) 
    #cam = norm_image(heatmap + np.float32(image[:,:,:3]))

    return image, cam, heatmap

def scale_cam_image(cam, target_size=None):
    result = []
    for img in cam:
        img = img - np.min(img)
        img = img / (1e-7 + np.max(img))
        if target_size is not None:
            img = cv2.resize(img, target_size)
        result.append(img)
    result = np.float32(result)

    return result

def norm_image(image): 
    """
    normalize image
    :param image: [H,W,C]
    :return:
    """
    image = image.copy()
    image -= np.max(np.min(image), 0)
    image /= np.max(image)
    image *= 255.
    return np.uint8(image)


def gen_gb(grad):
    """
    guided back propagation 
    :param grad: tensor,[3,H,W]
    :return:
    """
    # normalize
    grad = grad.data.numpy()
    gb = np.transpose(grad, (1, 2, 0))
    return gb

def save_image(image_dicts, input_image_name, network, output_dir):
    prefix = os.path.splitext(input_image_name)[0]
    for key, image in image_dicts.items():
        io.imsave(os.path.join(output_dir, '{}-{}-{}.jpg'.format(prefix, network, key)), image[:,:,0:3])


def main(args):
    # input
    #img = io.imread(args.image_path)
    r = np.load('./images_npy/r/' + args.image_path)
    b = np.load('./images_npy/b/' + args.image_path)
    g = np.load('./images_npy/g/' + args.image_path)
    nir = np.load('./images_npy/nir/' + args.image_path)
    #label_load = np.load('./images_npy/' + args.image_path)
    #label = (label_load > 0.0).astype(np.float32)
    img = np.moveaxis(np.concatenate([r,g,b,nir]).astype('float32'), 0, -1)
    img = cv2.resize(img, (256,256), interpolation=cv2.INTER_AREA)  
    '''
    startx = r.shape[2] // 2 - (256 // 2)
    starty = r.shape[1] // 2 - (256 // 2)
    img = np.moveaxis(np.concatenate([
            r[:, starty:starty+256, startx:startx+256],
            g[:, starty:starty+256, startx:startx+256],
            b[:, starty:starty+256, startx:startx+256],
            nir[:, starty:starty+256, startx:startx+256],
        ]).astype('float32'), 0, -1)
    '''
    #alpha = 
    inputs = prepare_input(img)
    # output 
    image_dict = {}
    # net
    net = get_net(args.network, args.weight_path)
    # Grad-CAM
    layer_name = get_last_conv_name(net) if args.layer_name is None else args.layer_name
    grad_cam = GradCAM(net, layer_name)
    mask = grad_cam(inputs, args.class_id)  # cam mask
    image_dict['orig'], image_dict['cam'], image_dict['heatmap'] = gen_cam(img, mask)
    grad_cam.remove_handlers()


    # GuidedBackPropagation
    gbp = GuidedBackPropagation(net)
    inputs.grad.zero_()  
    grad = gbp(inputs)

    gb = gen_gb(grad)
    image_dict['gb'] = norm_image(gb)
    # generate Guided Grad-CAM
    cam_gb = gb * mask[..., np.newaxis]
    image_dict['cam_gb'] = norm_image(cam_gb)

    save_image(image_dict, os.path.basename(args.image_path), args.network, args.output_dir)


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--network', type=str, default='resnet50',
                        help='ImageNet classification network')
    parser.add_argument('--image-path', type=str, default='L1470924.npy',
                        help='input image path')
    parser.add_argument('--weight-path', type=str, default='checkpoint_498_240.pth',
                        help='weight path of the model') #default='checkpoint_200_214.pth'
    parser.add_argument('--layer-name', type=str, default=None,
                        help='last convolutional layer name')
    parser.add_argument('--class-id', type=int, default=None,
                        help='class id')
    parser.add_argument('--output-dir', type=str, default='results',
                        help='output directory to save results')
    arguments = parser.parse_args(args=[])
    main(arguments)

layer4.2.conv3
feature shape:torch.Size([1, 2048, 8, 8])


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:133: UserWarning: results/L1470924-resnet50-gb.jpg is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:133: UserWarning: results/L1470924-resnet50-cam_gb.jpg is a low contrast image
